In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/
!unzip /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/StarGAN.zip

/content
Archive:  /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/StarGAN.zip
  inflating: logger.py               
  inflating: main.py                 
  inflating: model.py                
  inflating: solver.py               
  inflating: trai.jpg                
  inflating: data_loader.py          
  inflating: download.sh             
  inflating: gai.jpg                 
  inflating: haarcascade_frontalface_default.xml  


## Download dataset

In [3]:
#run this line 1 time only to downloading the data celebA
!bash download.sh celeba

Streaming output truncated to the last 5000 lines.
  inflating: ./data/celeba/images/195339.jpg  
  inflating: ./data/celeba/images/012465.jpg  
  inflating: ./data/celeba/images/089057.jpg  
  inflating: ./data/celeba/images/147796.jpg  
  inflating: ./data/celeba/images/192463.jpg  
  inflating: ./data/celeba/images/142818.jpg  
  inflating: ./data/celeba/images/148061.jpg  
  inflating: ./data/celeba/images/184432.jpg  
  inflating: ./data/celeba/images/201157.jpg  
  inflating: ./data/celeba/images/097288.jpg  
  inflating: ./data/celeba/images/130371.jpg  
  inflating: ./data/celeba/images/082864.jpg  
  inflating: ./data/celeba/images/017429.jpg  
  inflating: ./data/celeba/images/186077.jpg  
  inflating: ./data/celeba/images/126843.jpg  
  inflating: ./data/celeba/images/200381.jpg  
  inflating: ./data/celeba/images/012555.jpg  
  inflating: ./data/celeba/images/092238.jpg  
  inflating: ./data/celeba/images/008151.jpg  
  inflating: ./data/celeba/images/171418.jpg  
  inflati

## run train

In [ ]:
# Train StarGAN using the CelebA dataset
#num_iters#number of total iterations for training D
#num_iters_decay#number of iterations for decaying 
#n_critic#number of D updates per each G update
#c_dim#attrs number
%cd /content
!python main.py --mode train \
                --dataset CelebA \
                --image_size 256 \
                --celeba_crop_size 178 \
                --batch_size 16 \
                --beta1 0.5 \
                --beta2 0.99 \
                --c_dim 3 \
                --log_step 100 \
                --num_iters 100000 \
                --resume_iters 67000 \
                --num_iters_decay 50000 \
                --model_save_step 1000 \
                --sample_step 500 \
                --lr_update_step 500 \
                --g_lr 0.0001 \
                --d_lr 0.0001 \
                --n_critic 5 \
                --num_workers 2 \
                --sample_dir /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/samples \
                --log_dir /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/logs \
                --model_save_dir /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/models \
                --result_dir /content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/results \
                --selected_attrs Wearing_Lipstick No_Beard Male
#train6
#--dataset CelebA \
#--image_size 256 \
#--celeba_crop_size 178 \
#--batch_size 16 \
#--beta1 0.5 \
#--beta2 0.99 \
#--c_dim 3 \
#--log_step 100 \
#--num_iters 100000 \
#--resume_iters 54000 \
#--num_iters_decay 50000 \
#--model_save_step 1000 \
#--sample_step 500 \
#--lr_update_step 500 \
#--g_lr 0.0001 \
#--d_lr 0.0001 \
#--n_critic 5 \
#--num_workers 2 \
#--selected_attrs Wearing_Lipstick No_Beard Male

/content
Namespace(attr_path='data/celeba/list_attr_celeba.txt', batch_size=16, beta1=0.5, beta2=0.99, c2_dim=8, c_dim=3, celeba_crop_size=178, celeba_image_dir='data/celeba/images', d_conv_dim=64, d_lr=0.0001, d_repeat_num=6, dataset='CelebA', g_conv_dim=64, g_lr=0.0001, g_repeat_num=6, image_size=256, lambda_cls=1, lambda_gp=10, lambda_rec=10, log_dir='/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/logs', log_step=100, lr_update_step=500, mode='train', model_save_dir='/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/models', model_save_step=1000, n_critic=5, num_iters=100000, num_iters_decay=50000, num_workers=2, rafd_crop_size=256, rafd_image_dir='data/RaFD/train', result_dir='/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/results', resume_iters=67000, sample_dir='/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/samples', sample_step=500, selected_attrs=['Wearing_Lipst

# use model

In [ ]:
import random
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import cv2
%matplotlib inline

from model import Generator
from model import Discriminator

## Config


In [ ]:
dataset ='CelebA'
image_size =256
selected_attrs =['Wearing_Lipstick', 'No_Beard', 'Male']
# Dimension of domain labels (1st dataset, i.e. CelebA's selected attrs).
c_dim =3
celeba_image_dir = '/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/results'
model_save_dir ='/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/models'
# Number of conv filters in the first layer of G.
g_conv_dim = 64
# number of residual blocks in G
g_repeat_num = 6
# test model from this step
#test_iters = 100000

## Load model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
G = Generator(conv_dim=g_conv_dim, c_dim=c_dim, repeat_num=g_repeat_num)
D = Discriminator(image_size=image_size,conv_dim=g_conv_dim, c_dim=c_dim, repeat_num=g_repeat_num)

In [ ]:
def print_network(model, name):
    """Print out the network information."""
    num_params = 0
    for p in model.parameters():
        num_params += p.numel()
    print(model)
    print(name)
    print("The number of parameters: {}".format(num_params))

In [ ]:
print_network(G, 'G')

In [ ]:
print_network(D, 'D')

In [ ]:
G.to(device)
print('Models moved to', device)
D.to(device)
print('Models moved to', device)

Models moved to cuda
Models moved to cuda


# download pretrained-models

In [ ]:
%cd /content
!bash download.sh pretrained-celeba-128x128

In [ ]:
%cd /content
!bash download.sh pretrained-celeba-256x256

# load trained model:
## G model & D model

In [ ]:
#resume_iters = test_iters
#print('Loading the trained models from step {}...'.format(resume_iters))
#G_path = os.path.join(model_save_dir, '{}-G.ckpt'.format(resume_iters))
#D_path = os.path.join(model_save_dir, '{}-D.ckpt'.format(resume_iters))

G_path = '/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/models/39000-G.ckpt'
D_path = '/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/models/39000-D.ckpt'
#G_path = '/content/stargan_celeba_256/models/200000-G.ckpt'
#D_path = '/content/stargan_celeba_256/models/200000-D.ckpt'
G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

<All keys matched successfully>

## load face_cascade in cv2

In [ ]:
CASE_PATH = '/content/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(CASE_PATH)

In [ ]:
def crop_face(imgarray, section, margin=20, size=256):
    """
    :param imgarray: full image
    :param section: face detected area (x, y, w, h)
    :param margin: add some margin to the face detected area to include a full head
    :param size: the result image resolution with be (size x size)
    :return: resized image in numpy array with shape (size x size x 3)
    """
    img_h, img_w, _ = imgarray.shape
    if section is None:
        section = [0, 0, img_w, img_h]
    (x, y, w, h) = section
    margin = int(min(w, h) * margin / 100)
    x_a = x - margin
    y_a = y - margin
    x_b = x + w + margin
    y_b = y + h + margin
    if x_a < 0:
        x_b = min(x_b - x_a, img_w - 1)
        x_a = 0
    if y_a < 0:
        y_b = min(y_b - y_a, img_h - 1)
        y_a = 0
    if x_b > img_w:
        x_a = max(x_a - (x_b - img_w), 0)
        x_b = img_w
    if y_b > img_h:
        y_a = max(y_a - (y_b - img_h), 0)
        y_b = img_h
    cropped = imgarray[y_a: y_b, x_a: x_b]
    resized_img = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    resized_img = np.array(resized_img)
    return resized_img

## Generator

In [ ]:
from google.colab import files
# seed the pseudorandom number generator
from random import seed
from random import random
#plt.savefig("abc.png")
test_dir = '/content/'
results_dir =  '/content/drive/MyDrive/Nhom05_DeepLearning_GenderSwap/Project3/3d_256_178_16/results'
def tryCvStarGAN(imgFile, setLipMakeUp = False,setNoBeard=False, setMale = False):
    frame = cv2.imread(test_dir+imgFile)
    frame = frame[:, :, ::-1] # CV2'BGR -> RGB
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10
    )
    if len(faces) is 0:
        print('No face detected, try another image.')
        return
    main_face = max(faces, key=lambda rectangle: (rectangle[2] * rectangle[3]))  # area = w * h
    face_img = crop_face(frame, main_face, margin = 40, size=256)
    image = Image.fromarray(face_img)
    
    # Pre-process the image
    transform = []
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    preprocessed_image = transform(face_img)
    
    # Show the pre-processed source image.
    # Prepare the plot
    fig=plt.figure(figsize=(8, 8))
    columns = 2
    rows = 1
    fig.add_subplot(rows, columns, 1)
    plt.imshow((np.moveaxis(preprocessed_image.numpy(),[0], [2])+1)/2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Pre-process the image
    
    labels = torch.zeros(c_dim)
    # Set the 'LipMakeUp' label.
    if setLipMakeUp is True:
        labels[0] = 1
    # Set the 'Beard' label.
    if setNoBeard is True:
        labels[1] = 1
    # Set the 'Male' label.
    if setMale is True:
        labels[2] = 1
    # Run the generator to generate the desired image with labels.
    print(preprocessed_image.unsqueeze(0).to(device).shape)
    #x_fake = G(x_real, c_trg)
    print(labels)
    generated = G(preprocessed_image.unsqueeze(0).to(device), labels.unsqueeze(0).to(device))
    print(generated.shape)
    #out_src, out_cls = D(x_fake)
    out_src, out_cls = D(preprocessed_image.unsqueeze(0).to(device))
    print(out_cls)
    out_src2, out_cls2 = D(generated)
    print(out_cls2)


    # Show the generated image.
    fig.add_subplot(rows, columns, 2)
    plt.imshow((np.moveaxis(generated.cpu().detach().numpy()[0],[0], [2])+1)/2)
    #save image
    seed(1)
    rand_idx = random()
    plt.savefig(f"{results_dir}/{imgFile}")
    print(f"{results_dir}/{imgFile} saved")
    plt.show()

## to Female

In [ ]:
def swapToFemale(imgPath):
    frame = cv2.imread(test_dir+imgPath)
    frame = frame[:, :, ::-1] # CV2'BGR -> RGB
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10
    )
    if len(faces) is 0:
        print('No face detected, try another image.')
        return
    main_face = max(faces, key=lambda rectangle: (rectangle[2] * rectangle[3]))  # area = w * h
    face_img = crop_face(frame, main_face, margin = 40, size=256)
    image = Image.fromarray(face_img)
    
    # Pre-process the image
    transform = []
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    preprocessed_image = transform(face_img)


    minScore = 1000
    # Three hair color to choose from.
    hairColors = ['Black_Hair', 'Blond_Hair', 'Brown_Hair']
    for i, color in enumerate(hairColors):
        for stat in range(0,2):
            labels = torch.zeros(c_dim)  #0 0 0 0 0
            labels[i] = 1
            labels[3] = 0
            labels[4] = stat
            generated = G(preprocessed_image.unsqueeze(0).to(device), labels.unsqueeze(0).to(device))
            out_src, out_cls = D(generated)
            maleScore = out_cls[0][3].item()
            if (maleScore<minScore):
                minScore = maleScore
                setHairColor = hairColors[i]
                if (stat==1):
                    setYoung = True
                else:
                    setYoung = False
    print('minScore= ',minScore)
    print('Swapping to female... set: setHairColor=',setHairColor, ' setYoung= ',setYoung)
    tryCvStarGAN(imgPath, setHairColor = setHairColor, setMale = False, setYoung = setYoung)

## to Male

In [ ]:
def swapToMale(imgPath):
    frame = cv2.imread(test_dir+imgPath)
    frame = frame[:, :, ::-1] # CV2'BGR -> RGB
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10
    )
    if len(faces) is 0:
        print('No face detected, try another image.')
        return
    main_face = max(faces, key=lambda rectangle: (rectangle[2] * rectangle[3]))  # area = w * h
    face_img = crop_face(frame, main_face, margin = 40, size=256)
    image = Image.fromarray(face_img)
    
    # Pre-process the image
    transform = []
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    preprocessed_image = transform(face_img)


    maxScore = -1000
    # Three hair color to choose from.
    hairColors = ['Black_Hair', 'Blond_Hair', 'Brown_Hair']
    for i, color in enumerate(hairColors):
        for stat in range(0,2):
            labels = torch.zeros(c_dim)  #0 0 0 0 0
            labels[i] = 1
            labels[3] = 1
            labels[4] = stat
            generated = G(preprocessed_image.unsqueeze(0).to(device), labels.unsqueeze(0).to(device))
            out_src, out_cls = D(generated)
            maleScore = out_cls[0][3].item()
            if (maleScore>maxScore):
                maxScore = maleScore
                setHairColor = hairColors[i]
                if (stat==1):
                    setYoung = True
                else:
                    setYoung = False
    print('maxScore= ',maxScore)              
    print('Swapping to male... set: setHairColor=',setHairColor, ' setYoung= ',setYoung)
    tryCvStarGAN(imgPath, setHairColor = setHairColor, setMale = True, setYoung = setYoung)

## swap

In [ ]:
def swapGender(imgPath):
    frame = cv2.imread(test_dir+imgPath)
    frame = frame[:, :, ::-1] # CV2'BGR -> RGB
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10
    )
    if len(faces) is 0:
        print('No face detected, try another image.')
        return
    main_face = max(faces, key=lambda rectangle: (rectangle[2] * rectangle[3]))  # area = w * h
    face_img = crop_face(frame, main_face, margin = 40, size=256)
    image = Image.fromarray(face_img)
    
    # Pre-process the image
    transform = []
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    preprocessed_image = transform(face_img)

    out_src, out_cls = D(preprocessed_image.unsqueeze(0).to(device))
    print(out_cls.shape)
    print(out_cls)
    maleScore = out_cls[0][3].item()
    print(maleScore)
    if (maleScore > 0 ):
        print('Male detected with maleScore= ',maleScore)
        swapToFemale(imgPath)
    else:
        print('Female detected with maleScore= ',maleScore)
        swapToMale(imgPath)

#Time to see some results

In [ ]:
swapGender("trai.jpg")

In [ ]:
swapGender("gai.jpg")

In [ ]:
print("label: lip/beard/male")
tryCvStarGAN('man5.png', setLipMakeUp = True, setNoBeard=True, setMale = False)

In [ ]:
print("label: lip/beard/male")
tryCvStarGAN('gai.jpg', setLipMakeUp = True, setNoBeard=True, setMale = False)